# Create dice objects to be used in simple dice game simulations

In [13]:
import numpy as np

class Die:
    '''
    Dice have a number of sides and, unless in motion, always have one face up.  
    
    Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
    '''
    
    def __init__(self,sides):
        self.sides = sides
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
    def roll(self):
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
        return self.face_up

In [14]:
die = Die(sides=6)

In [3]:
die.face_up

2

In [4]:
die.roll()

5

In [5]:
die.face_up

5

In [45]:
import numpy as np
from collections import Counter

class Dice_Game:
    '''
    This simple multiplayer dice game is the Beebe family's variation of a popular dice game, we just call it "Dice".  
    
    Instructions:
    
    Players determine who goes first, and proceed taking turns in an order (typically clockwise around a table).
    
    Each player, in turn, proceeds by first shaking all six dice, 
    and proceeding along a decision tree of subsequent rolls.
    This tree ends in either a zero score, where a shake results in no dice count for points, 
    or the player "keeps" a number of dice which count for points.
    
    The point value of dice are determined per shake, 
    and point values do not change when mixed with dice kept from previous rolls.
    
    COUNTERS:
    
    One: 100
    Five: 50
    
    THREE-OF-A-KIND:
    Ones: 1000
    Sixes: 600
    Fives: 500
    Fours: 400
    Threes: 300
    Twos: 200
    
    STRAIGHT (1,2,3,4,5,6): 2500
    
    FLUSH (ALL-OF-A-KIND): Instant Win.
    
    On the occasion where all dice rolled are "counters", the dice must all be brought back and shook again.
    The turn proceeds as usual, adding the new shake points to the previous all-counter results.
    
    
    '''
    
    def __init__(self, players, num_dice=6):
        self.player_names = ['Player_{}'.format(name + 1) for name in [j for j in range(players)]]
        self.players = [self.Player(name=player) for player in self.player_names]
        self.dice = [self.Die(sides=6) for die in [i for i in range(num_dice)]]
        self.play_to = 7500
        self.winner = False
        self.turn = 0
        
        
    def start():
        self.score = score
        
    
    def shake(self,dice_to_roll,Player):
        shook_dice = [die.roll() for die in dice_to_roll]
        print('I,', Player.name, 'shook the dice:', [i.face_up for i in dice_to_roll])
        print('TEST FUNC I,', Player.name, 'shook the dice:', object_list_to_faces(dice_to_roll))
        #return shook_dice
        return dice_to_roll

    def object_list_to_faces(die_list):
        '''
        Function takes list (potentially including lists) of die objects and returns a list (of lists) of the die faces.
        '''

        face_list = [i.face_up if type(i) == Dice_Game.Die else [j.face_up for j in i] for i in die_list]

        return face_list
        
    def player_take_turn(self,Player):
        Player.turn += 1
        still_to_shake = self.dice.copy()
        #kept = []
        Player.turn_shakes = 0
        #print('length of self.dice', len(self.dice))
        while len(still_to_shake) > 0:
            if Player.turn_shakes == 0:
                print('lets goooo!')
                
                shake = self.shake(still_to_shake,Player)
                Player.turn_shakes += 1
                shake_faces = [i.face_up for i in shake]
                #print('shake:',shake_faces)
                counters, bring_back = self.detect_keepable(shake)
                kept, still_to_shake = self.keep_dice(shake, counters)
                #print('objects still to shake:',still_to_shake)
                print('still to shake faces:', [i.face_up for i in still_to_shake])
                print('TEST FUNC still to shake faces:', object_list_to_faces(still_to_shake))
                #print('TO KEEP:',to_keep)
                #kept.append(to_keep)
                Player.learning_history.extend(kept)
                #print('KEPT:',kept)
                print('FIRST JUNCTION')
                print('On turn',Player.turn,'shake',Player.turn_shakes,', I,',Player.name,'kept', [i.face_up if type(i) != list else [j.face_up for j in i] for i in kept],'and my total kept dice is', [i.face_up if type(i) != list else [j.face_up for j in i] for i in kept])
                print('TEST FUNC On turn',Player.turn,'shake',Player.turn_shakes,', I,',Player.name,'kept', object_list_to_faces(kept),'and my total kept dice is', object_list_to_faces(kept))

                if bring_back:
                    print('bringing back from first junction.............')
                    still_to_shake = self.dice.copy()
                
            elif kept != [] and Player.turn_shakes > 0:
                print("keep 'em rollin'!")
                print('kept so far:', )
                
                shake = self.shake(still_to_shake,Player)
                Player.turn_shakes += 1
                shake_faces = [i.face_up for i in shake]
                print('shake:',shake_faces)
                counters, bring_back = self.detect_keepable(shake)
                to_keep, still_to_shake = self.keep_dice(shake, counters)
                #print('objects still to shake:',still_to_shake)
                #print('still to shake faces:', [i.face_up for i in still_to_shake])
                print('TO KEEP:',[i.face_up if type(i) == Dice_Game.Die else [j.face_up for j in i] for i in to_keep])
                print('TEST FUNC TO KEEP:',object_list_to_faces(to_keep))
                kept.extend(to_keep)
                Player.learning_history.append(kept)
                #print('KEPT:',kept)
                print('SECOND JUNCTION')
                print('On turn',Player.turn,'shake',Player.turn_shakes,', I,',Player.name,'kept', [i.face_up if type(i) != list else [j.face_up for j in i] for i in to_keep],'and my total kept dice is', [i.face_up if type(i) != list else [j.face_up for j in i] for i in kept])
                print('TEST FUNC On turn',Player.turn,'shake',Player.turn_shakes,', I,',Player.name,'kept', object_list_to_faces(to_keep),'and my total kept dice is', object_list_to_faces(kept))
                if bring_back:
                    print('bringing back from second junction.............')
                    still_to_shake = self.dice.copy()
                    
            elif kept == [] and Player.turn_shakes > 0:
                print("you got nuthin:'",shake_faces, ". next player's turn.")
                break
        
        
        
        
        
    def detect_keepable(self, shook_dice):
        
        '''
        Method which sorts through the shook dice and returns a list (of lists) of potentially keepable dice.
        
        For example, if the dice objects shook show [1,2,5,4,4,4] detect_keepable will return: [[4,4,4],1,5]
        
        '''
        print('BEGIN DETECTING KEEPABLES')
        
        BRING_EM_BACK = 0
        
        #print('shook dice:', shook_dice)
        
        dice_faces = [i.face_up for i in shook_dice]
        #print('dice faces', dice_faces)
        
        three_of_a_kinds = [i for i in Counter(dice_faces) if Counter(dice_faces)[i] >= 3]
        print('three_of_a_kinds:', three_of_a_kinds)
        
        keepable_three_of_a_kinds = []
        for set_of_three in three_of_a_kinds:
            print('set_of_three',set_of_three)
            toak = []
            for i in shook_dice:
                if i.face_up == set_of_three and len(toak) <= 2:
                    toak.append(i)
            keepable_three_of_a_kinds.append(toak)
            #keepable_three_of_a_kinds.append([i for i in shook_dice if i.face_up == set_of_three])
            #print('appended keepable three of a kind objects:', keepable_three_of_a_kinds)

        keepable = keepable_three_of_a_kinds
        #print('keepable objects after adding three of a kinds:', keepable)
        print('keepable dice faces after adding three of a kinds:', [j.face_up for i in keepable for j in i])

        
        for i in shook_dice:
            if i.face_up == 1 or i.face_up == 5:
                keepable.append(i)
        #print('keepable objects after adding ones and fives:', keepable)
        print('keepable dice faces after adding ones and fives:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in keepable])
                

        if set(dice_faces) == set([1,2,3,4,5,6]):
            keepable.append(shook_dice)
            #print('keepable objects after adding straight:', keepable)
            print('keepable dice faces after adding straight:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in keepable])

        print('total keepable list of dice faces:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in keepable])
        
        if all(die in keepable for die in shook_dice):
            print('total keepable list of dice faces:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in keepable])
            print('ALL COUNTERS.  BRING EM BACK!  NEED FUNCTION HERE!!!!')
            BRING_EM_BACK = 1
        
        return keepable, BRING_EM_BACK
    
    
    def keep_dice(self, shook_dice, keepable_list):
        '''
        Method for choosing which dice to keep.  
        
        This will in particular need to be interfaced for agent to learn.
        
        '''
        
        print('BEGIN KEEPING DICE')
        
        #print('keepable object list:', keepable_list)
        print('keepable dice faces list:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in keepable_list])
        print('len keepable_list:', len(keepable_list))
        if len(keepable_list) is 0 or len(keepable_list) is 1:
            kept_dice = keepable_list
        else:
            kept_dice = [np.random.choice(keepable_list)]
        #print('kept dice objects:', kept_dice)
        print('kept dice faces:', [i.face_up if type(i) != list else [j.face_up for j in i] for i in kept_dice])
        

        for i in kept_dice:
            #print('object i in kept_dice:', i)
            if type(i) is list:
                print('i in kept_dice is list:', [j.face_up for j in i])
                for j in i:
                    #print('j object in list i:', j)
                    print('j.face_up in list i:', j.face_up)
                    shook_dice.remove(j)
                    #print('shook_dice removed j object:',shook_dice)
                    print('remaining shook_dice faces after removing j:', [i.face_up for i in shook_dice])
            elif i in shook_dice:
                shook_dice.remove(i)

        dice_still_to_shake = shook_dice
        print('dice still to shake faces:', [i.face_up for i in dice_still_to_shake])

        return kept_dice, dice_still_to_shake

    
    
    class Die:
        '''
        Dice have a number of sides and, unless in motion, always have one face up.  

        Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
        '''

        def __init__(self,sides):
            self.sides = sides
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

        def roll(self):
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

            return self.face_up
    
    # Players as subclass, called in Dice_Game init
    class Player:
        '''
        Each player is equipped with several attributes, mainly for statistics and for future agent-based learning.
        '''
    
        def __init__(self,name):
            self.name = name
            self.turn = 0
            self.score = 0
            self.learning_history = []
            self.turn_shakes = 0
            self.game_shakes = 0

            


In [46]:
game = Dice_Game(players=3)

In [47]:
game.play_to

7500

In [48]:
print([i.face_up for i in game.dice])

[2, 1, 4, 2, 1, 3]


In [49]:
game.players[0].name

'Player_1'

In [50]:
shake = game.shake(game.dice,game.players[0])

I, Player_1 shook the dice: [3, 1, 3, 2, 1, 1]
TEST FUNC I, Player_1 shook the dice: [3, 1, 3, 2, 1, 1]


In [51]:
game.player_take_turn(game.players[0])

lets goooo!
I, Player_1 shook the dice: [4, 5, 4, 4, 4, 1]
TEST FUNC I, Player_1 shook the dice: [4, 5, 4, 4, 4, 1]
BEGIN DETECTING KEEPABLES
three_of_a_kinds: [4]
set_of_three 4
keepable dice faces after adding three of a kinds: [4, 4, 4]
keepable dice faces after adding ones and fives: [[4, 4, 4], 5, 1]
total keepable list of dice faces: [[4, 4, 4], 5, 1]
BEGIN KEEPING DICE
keepable dice faces list: [[4, 4, 4], 5, 1]
len keepable_list: 3
kept dice faces: [5]
dice still to shake faces: [4, 4, 4, 4, 1]
still to shake faces: [4, 4, 4, 4, 1]
TEST FUNC still to shake faces: [4, 4, 4, 4, 1]
FIRST JUNCTION
On turn 1 shake 1 , I, Player_1 kept [5] and my total kept dice is [5]
TEST FUNC On turn 1 shake 1 , I, Player_1 kept [5] and my total kept dice is [5]
keep 'em rollin'!
kept so far:
I, Player_1 shook the dice: [6, 3, 3, 1, 2]
TEST FUNC I, Player_1 shook the dice: [6, 3, 3, 1, 2]
shake: [6, 3, 3, 1, 2]
BEGIN DETECTING KEEPABLES
three_of_a_kinds: []
keepable dice faces after adding three o

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:209: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



TEST FUNC TO KEEP: []
SECOND JUNCTION
On turn 1 shake 33 , I, Player_1 kept [] and my total kept dice is [6, 3, 3, 5, 3, 4, 4, 3, 3, 3, 6, 5, 3, 6, 5, 4, 3, 3, 6, 5, 4, 3, 3, 3, [3, 3, 3], 5]
TEST FUNC On turn 1 shake 33 , I, Player_1 kept [] and my total kept dice is [6, 3, 3, 5, 3, 4, 4, 3, 3, 3, 6, 5, 3, 6, 5, 4, 3, 3, 6, 5, 4, 3, 3, 3, [3, 3, 3], 5]
keep 'em rollin'!
kept so far:
I, Player_1 shook the dice: [5, 5]
TEST FUNC I, Player_1 shook the dice: [5, 5]
shake: [5, 5]
BEGIN DETECTING KEEPABLES
three_of_a_kinds: []
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [5, 5]
total keepable list of dice faces: [5, 5]
total keepable list of dice faces: [5, 5]
ALL COUNTERS.  BRING EM BACK!  NEED FUNCTION HERE!!!!
BEGIN KEEPING DICE
keepable dice faces list: [5, 5]
len keepable_list: 2
kept dice faces: [5]
dice still to shake faces: [5]
TO KEEP: [5]
TEST FUNC TO KEEP: [5]
SECOND JUNCTION
On turn 1 shake 34 , I, Player_1 kept [5] and 

SECOND JUNCTION
On turn 1 shake 72 , I, Player_1 kept [] and my total kept dice is [1, 5, 4, 1, 6, 1, 1, 5, 4, 6, 1, 1, 4, 1, 1, 1, 6, 5, 1, 1, 1, 4, 6, 5, [6, 5, 4], 1, 1, 4, 1, 1, 5, 6, 1, 1, 4, 6, 1, 1, 5, 1, 6, 1, 5, 4, 1, 1, 1, 5, 1]
TEST FUNC On turn 1 shake 72 , I, Player_1 kept [] and my total kept dice is [1, 5, 4, 1, 6, 1, 1, 5, 4, 6, 1, 1, 4, 1, 1, 1, 6, 5, 1, 1, 1, 4, 6, 5, [6, 5, 4], 1, 1, 4, 1, 1, 5, 6, 1, 1, 4, 6, 1, 1, 5, 1, 6, 1, 5, 4, 1, 1, 1, 5, 1]
keep 'em rollin'!
kept so far:
I, Player_1 shook the dice: [6, 2]
TEST FUNC I, Player_1 shook the dice: [6, 2]
shake: [6, 2]
BEGIN DETECTING KEEPABLES
three_of_a_kinds: []
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: []
total keepable list of dice faces: []
BEGIN KEEPING DICE
keepable dice faces list: []
len keepable_list: 0
kept dice faces: []
dice still to shake faces: [6, 2]
TO KEEP: []
TEST FUNC TO KEEP: []
SECOND JUNCTION
On turn 1 shake 73 , I, Player_1 kept [

In [52]:
game.players[0].turn

1

In [ ]:
def kept_dice_to_potential_score(kept_dice):
    
    
    
    return potential_score

In [146]:
def stay_or_go(kept,Player):
    
    return decision

In [24]:
diex = Dice_Game.Die(6)

In [25]:
type(diex)

__main__.Dice_Game.Die

In [26]:
type(diex) == Dice_Game.Die

True

In [27]:
diex

In [28]:
diex.face_up

4

In [29]:
diey = Dice_Game.Die(6)
diey.face_up

4

In [30]:
diez = Dice_Game.Die(6)
diez.face_up

5

In [31]:
list_of_die = [diex,diey,diez]

In [33]:
list_of_faces = [diex.face_up,diey.face_up,diez.face_up]
list_of_faces

[4, 4, 5]

In [34]:
if diex in list_of_faces:
    print('h')

In [35]:
list_of_faces.extend([3,[3,4]])
list_of_faces

[4, 4, 5, 3, [3, 4]]

In [37]:
all(die.face_up in list_of_faces for die in list_of_die)

True

In [38]:
print(die.face_up in list_of_faces for die in list_of_die)

<generator object <genexpr> at 0x11e63a250>


In [30]:
def object_list_to_faces(die_list):
    '''
    Function takes list (potentially including lists) of die objects and returns a list (of lists) of the die faces.
    '''
    
    face_list = [i.face_up if type(i) == Dice_Game.Die else [j.face_up for j in i] for i in die_list]
    
    return face_list
    